# Problem Statement
Fine tune the "microsoft/phi-2" model to generate customized response to the customer queries.

# Install Required Packages

In [1]:
!pip install datasets>=4.4.1 transformers>=4.57.3 peft>=0.18.0 evaluate>=0.4.6 bitsandbytes>=0.48.2 huggingface_hub python-dotenv>=1.0.1

# Import Package and Load ENV variables

In [2]:
import os
import torch
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from dotenv import load_dotenv
from huggingface_hub import login

In [3]:
import os

# Detect if running in Google Colab
def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    from google.colab import userdata
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN_WRITE")
else:
    # Running locally → load from .env
    from dotenv import load_dotenv
    load_dotenv()  # loads variables from .env into environment
    os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")


In [4]:
import sys
print(sys.executable)

/usr/bin/python3


In [5]:
# -----------------------------
# Device & dtype setup
# -----------------------------
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

print(f"Using device: {device} with dtype: {torch_dtype}")

Using device: cuda with dtype: torch.float16


# Prepare Dataset for Training

In [6]:
# -----------------------------
# Synthetic dataset
# -----------------------------
instructions = [
    "Customer asks about refund window",
    "Customer wants to cancel an order",
    "Order arrived late",
    "Wrong item received",
    "Product not working",
    "Shipping cost inquiry",
    "Change delivery address",
    "Request for invoice",
    "Ask about warranty",
    "Technical support request"
]

responses = [
    "Our refund window is 30 days from delivery.",
    "You can cancel your order from your account dashboard within 24 hours.",
    "Sorry for the delay. A delivery credit has been applied.",
    "We’ll ship the correct item and provide a return label.",
    "Please try resetting the product. Contact support if the issue persists.",
    "Shipping cost depends on your location and chosen delivery speed.",
    "You can update your delivery address before the order ships.",
    "An invoice will be emailed to you after purchase.",
    "Your product comes with a 12-month warranty.",
    "Our tech support team will contact you shortly."
]

train_data = [
    {"instruction": f"{instructions[random.randint(0,len(instructions)-1)]} #{i+1}",
     "response": responses[random.randint(0,len(responses)-1)]}
    for i in range(300)
]

dataset = Dataset.from_list(train_data)

In [7]:
import pandas as pd
from datasets import Dataset

# -----------------------------
# Load dataset from CSV
# -----------------------------
# csv_path = "/content/drive/MyDrive/buildllm/_data/customer_response_data.csv"  # path to your CSV file
csv_path = "https://huggingface.co/datasets/mishrabp/customer-support-responses/resolve/main/train.csv"

# Read CSV with pandas
df = pd.read_csv(csv_path)

df.head(5)

# Ensure columns are named 'instruction' and 'response'
assert "instruction" in df.columns and "response" in df.columns, "CSV must have 'instruction' and 'response' columns"

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

print("Number of training examples:", len(dataset))
print("First example:", dataset[0])
print(df.head(5))

Number of training examples: 3000
First example: {'instruction': 'Customer wants to cancel an order #1', 'response': 'You can cancel your order from your account dashboard within 24 hours.'}
                            instruction  \
0  Customer wants to cancel an order #1   
1  Customer asks about refund window #2   
2               Payment failed issue #3   
3           Request replacement item #4   
4                  Track my shipment #5   

                                            response  
0  You can cancel your order from your account da...  
1        Our refund window is 30 days from delivery.  
2  Your payment did not go through. Please try ag...  
3  We will send a replacement immediately and sha...  
4  You can track your shipment using the tracking...  


# Load the Base Model

In [8]:
# -----------------------------
# Model + Tokenizer
# -----------------------------
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# QLoRA only if GPU available
bnb_config = None
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto" if device == "cuda" else None,
    torch_dtype=torch_dtype,
    quantization_config=bnb_config
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Preprocessing Data

In [9]:
# -----------------------------
# Preprocessing
# -----------------------------
def preprocess(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    enc = tokenizer(prompt, padding="max_length", truncation=True, max_length=256)
    enc["labels"] = enc["input_ids"].copy()
    return enc

tokenized_dataset = dataset.map(preprocess)
tokenized_dataset.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

# Configure LoRA and Quantize

In [10]:
# -----------------------------
# LoRA config
# -----------------------------
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],  # adjust for phi-2 if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, lora_cfg)

# Train Model

In [11]:
# -----------------------------
# Training arguments
# -----------------------------
training_args = TrainingArguments(
    output_dir="./outputs",
    learning_rate=2e-4,
    per_device_train_batch_size=1 if device=="cpu" else 2,
    num_train_epochs=1,
    logging_steps=5,
    save_strategy="no",
    report_to="none",
    fp16=True if device=="cuda" else False
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)
trainer.train()

Step,Training Loss
5,6.945700
10,6.150300
15,4.240200
20,1.284500
25,0.557900
30,0.481200
35,0.414800
40,0.349900
45,0.308400
50,0.271800


TrainOutput(global_step=1500, training_loss=0.1179121265411377, metrics={'train_runtime': 485.8407, 'train_samples_per_second': 6.175, 'train_steps_per_second': 3.087, 'total_flos': 1.221688295424e+16, 'train_loss': 0.1179121265411377, 'epoch': 1.0})

# Upload QLoRA Adapater and README.md to Hugging Face

In [17]:
# -----------------------------
# Save & push adapters to Hugging Face
# -----------------------------
model.save_pretrained("./phi2-custom-response-qlora-adapter")
tokenizer.save_pretrained("./phi2-custom-response-qlora-adapter")

model.push_to_hub("mishrabp/phi2-custom-response-qlora-adapter", use_auth_token=os.environ["HF_TOKEN"])
tokenizer.push_to_hub("mishrabp/phi2-custom-response-qlora-adapter", use_auth_token=os.environ["HF_TOKEN"])

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########| 10.5MB / 10.5MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mishrabp/phi2-custom-response-qlora-adapter/commit/8bc4218eddd3fea997c88e6f5e4348327cf98345', commit_message='Upload tokenizer', commit_description='', oid='8bc4218eddd3fea997c88e6f5e4348327cf98345', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mishrabp/phi2-custom-response-qlora-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='mishrabp/phi2-custom-response-qlora-adapter'), pr_revision=None, pr_num=None)

In [39]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN_WRITE")

readme_content = """
---
license: mit
tags:
  - causal-lm
  - instruction-following
  - loRA
  - QLoRA
  - quantized
language: en
library_name: transformers
base_model: microsoft/phi-2
---

# Phi-2 QLoRA Fine-Tuned Model


**Model:** `mishrabp/phi2-custom-response-qlora-adapter`

**Base Model:** [`microsoft/phi-2`](https://huggingface.co/microsoft/phi-2)

**Fine-Tuning Method:** QLoRA (4-bit quantized LoRA)

**Task:** Instruction-following / Customer Support Responses

---

## Model Description

This repository contains a **Phi-2 language model fine-tuned using QLoRA** on a synthetic dataset of customer support instructions and responses. The fine-tuning uses **4-bit quantized LoRA adapters** for memory-efficient training and can run on GPU or CPU (slower on CPU).

The model is designed for **instruction-following tasks** like customer support, FAQs, or other dialog generation tasks.

---

## Training Data

The fine-tuning dataset is synthetic, consisting of 3000 instruction-response pairs:

**Example:**

```text
Instruction: "Customer asks about refund window #1"
Response: "Our refund window is 30 days from delivery."
```

Here is the dataset that was used for fine-tunning:
https://huggingface.co/datasets/mishrabp/customer-support-responses/resolve/main/train.csv

You can replace the dataset with your own CSV/JSON file to train on real-world data.

---

## Intended Use

* Generate responses to instructions in customer support scenarios.
* Small-scale instruction-following experiments.
* Educational or research purposes.

---

## How to Use

### Load the Fine-Tuned Model

```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -----------------------------
# Load fine-tuned model from HF
# -----------------------------
model_name = "mishrabp/phi2-custom-response-qlora-adapter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
model = PeftModel.from_pretrained(base_model, model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# -----------------------------
# Sample evaluation dataset
# -----------------------------
eval_data = [
    {"instruction": "Customer asks about refund window", "reference": "Our refund window is 30 days from delivery."},
    {"instruction": "Order arrived late", "reference": "Sorry for the delay. A delivery credit has been applied."},
    {"instruction": "Wrong item received", "reference": "We’ll ship the correct item and provide a return label."},
]

# -----------------------------
# Evaluation loop
# -----------------------------
for i, example in enumerate(eval_data, 1):
    prompt = f"### Instruction:\\n{example['instruction']}\\n\\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_new_tokens=50)
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"Example {i}")
    print("Instruction:", example["instruction"])
    print("Generated Response:", generated.split("### Response:")[-1].strip())
    print("Reference Response:", example["reference"])
    print("-" * 50)

# -----------------------------
# Optional: compute simple token-level accuracy or BLEU
# -----------------------------
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = []
for example in eval_data:
    prompt = f"### Instruction:\\n{example['instruction']}\\n\\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_new_tokens=50)
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True).split("### Response:")[-1].strip()

    reference_tokens = example["reference"].split()
    generated_tokens = generated.split()
    bleu = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores.append(bleu)

print("Average BLEU score:", sum(bleu_scores)/len(bleu_scores))



```

---

## Training Script

The training script performs the following steps:

1. Loads the **Phi-2 base model**.
2. Creates a **synthetic dataset** of instruction-response pairs.
3. Tokenizes and formats the dataset for causal language modeling.
4. Applies a **LoRA adapter**.
5. Trains using **QLoRA** if GPU is available, otherwise full-precision LoRA on CPU.
6. Saves the adapter and tokenizer to `./phi2-qlora`.
7. Pushes the adapter and tokenizer to Hugging Face Hub.

### Requirements

```bash
pip install torch transformers peft datasets huggingface_hub python-dotenv
```

---

## Parameters

* `r=8`, `lora_alpha=16`, `lora_dropout=0.05`
* `target_modules=["q_proj","v_proj"]` (adjust for different base models)
* Learning rate: `2e-4`
* Batch si

"""

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

from huggingface_hub import HfApi, Repository

repo_id = "mishrabp/phi2-custom-response-qlora-adapter"

# Option 1: Using HfApi to upload README
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",  # must be exactly README.md for HF Hub
    repo_id=repo_id,
    repo_type="model",
    token=os.environ["HF_TOKEN"]
)


CommitInfo(commit_url='https://huggingface.co/mishrabp/phi2-custom-response-qlora-adapter/commit/31e424e4e3f4a8aa872d787e64805640c3ea0e93', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='31e424e4e3f4a8aa872d787e64805640c3ea0e93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mishrabp/phi2-custom-response-qlora-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='mishrabp/phi2-custom-response-qlora-adapter'), pr_revision=None, pr_num=None)

# Inference the Model from Hugging Face Upload

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -----------------------------
# Load fine-tuned model from HF
# -----------------------------
model_name = "mishrabp/phi2-custom-response-qlora-adapter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
model = PeftModel.from_pretrained(base_model, model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# -----------------------------
# Sample evaluation dataset
# -----------------------------
eval_data = [
    {"instruction": "Customer asks about refund window", "reference": "Our refund window is 30 days from delivery."},
    {"instruction": "Order arrived late", "reference": "Sorry for the delay. A delivery credit has been applied."},
    {"instruction": "Wrong item received", "reference": "We’ll ship the correct item and provide a return label."},
]

# -----------------------------
# Evaluation loop
# -----------------------------
for i, example in enumerate(eval_data, 1):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_new_tokens=50)
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"Example {i}")
    print("Instruction:", example["instruction"])
    print("Generated Response:", generated.split("### Response:")[-1].strip())
    print("Reference Response:", example["reference"])
    print("-" * 50)

# -----------------------------
# Optional: compute simple token-level accuracy or BLEU
# -----------------------------
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = []
for example in eval_data:
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_new_tokens=50)
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True).split("### Response:")[-1].strip()

    reference_tokens = example["reference"].split()
    generated_tokens = generated.split()
    bleu = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores.append(bleu)

print("Average BLEU score:", sum(bleu_scores)/len(bleu_scores))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example 1
Instruction: Customer asks about refund window
Generated Response: Our refund window is 30 days from delivery.
Reference Response: Our refund window is 30 days from delivery.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example 2
Instruction: Order arrived late
Generated Response: Sorry for the delay. A delivery credit has been applied.
Reference Response: Sorry for the delay. A delivery credit has been applied.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example 3
Instruction: Wrong item received
Generated Response: We’ll ship the correct item and provide a return label.
Reference Response: We’ll ship the correct item and provide a return label.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Average BLEU score: 1.0


## 🧩 Merge LoRA Adapter to Base Model and Upload

In [24]:
# # -----------------------------
# # 1. Merge LoRA adapter with the base model
# # -----------------------------
# print("Starting merge of LoRA adapter to base model on CPU...")

# # Load base model, forcing it onto the CPU (device_map=None, no .to(device))
# base_model_full = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     # Load to CPU in full float32 precision
#     torch_dtype=torch.float32,
#     low_cpu_mem_usage=True,
# )
# # ❌ REMOVE: if device == "cuda": base_model_full = base_model_full.to(device)

# tokenizer_full = AutoTokenizer.from_pretrained(model_name)
# tokenizer_full.pad_token = tokenizer_full.eos_token

# # Load the fine-tuned adapter weights
# from peft import PeftModel
# adapter_path = "./phi2-custom-response-qlora-adapter"

# # Load the adapter onto the CPU-based base model
# merged_model = PeftModel.from_pretrained(base_model_full, adapter_path)

# # Merge the adapter weights into the base model weights on the CPU
# # This will be slow, but it avoids OOM.
# merged_model = merged_model.merge_and_unload()
# print("LoRA adapter merged successfully on CPU.")

# # -----------------------------
# # 2. Save and Upload the merged model
# # -----------------------------
# merged_model_dir = "./phi2-custom-response-merged"
# # Save the model and tokenizer from CPU memory
# merged_model.save_pretrained(merged_model_dir)
# tokenizer_full.save_pretrained(merged_model_dir)

Starting merge of LoRA adapter to base model on CPU...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LoRA adapter merged successfully on CPU.


UnboundLocalError: cannot access local variable 'active_adapters' where it is not associated with a value

## Upload the Merged Model to HF

In [ ]:
# # -----------------------------
# # 4. Upload README.md file to the merged repository
# # -----------------------------
# repo_id_merged = merged_repo_id # Use the new merged repo ID

# # Update the README content to point to the MERGED model
# readme_content_merged = readme_content.replace(
#     "mishrabp/phi2-custom-response-qlora-adapter",
#     merged_repo_id
# )

# with open("README_merged.md", "w", encoding="utf-8") as f:
#     f.write(readme_content_merged)

# from huggingface_hub import HfApi
# api = HfApi()

# # Upload the new README to the merged model repository
# api.upload_file(
#     path_or_fileobj="README_merged.md",
#     path_in_repo="README.md",
#     repo_id=repo_id_merged,
#     repo_type="model",
#     token=os.environ["HF_TOKEN"]
# )
# print("Merged model, tokenizer, and README uploaded successfully.")